In [2]:
import pandas as pd
import numpy as np

In [3]:
aucs = pd.read_csv('AUCs.txt', delimiter='\t')
aucs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  14 non-null     object 
 1   C4.5        14 non-null     float64
 2   C4.5+m      14 non-null     float64
 3   C4.5+cf     14 non-null     float64
 4   C4.5+m+cf   14 non-null     float64
dtypes: float64(4), object(1)
memory usage: 688.0+ bytes


In [4]:
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [8]:
from itertools import combinations
from scipy import stats

In [9]:
w_stat = pd.DataFrame(columns=['Model 1', 'Model 2', 'Wilcoxon stat', 'p-value'])
k = 0
for i, j in combinations([1, 2, 3, 4], 2):
    w_stat.loc[k, 'Model 1'], w_stat.loc[k, 'Model 2'] = aucs.columns[i], aucs.columns[j]
    w_stat.loc[k, 'Wilcoxon stat'], w_stat.loc[k, 'p-value'] = stats.wilcoxon(aucs.iloc[:, i], aucs.iloc[:, j])
    k += 1

In [10]:
w_stat

,Model 1,Model 2,Wilcoxon stat,p-value
0,C4.5,C4.5+m,6.5,0.0107571
1,C4.5,C4.5+cf,43,0.861262
2,C4.5,C4.5+m+cf,11,0.0159064
3,C4.5+m,C4.5+cf,17,0.0463327
4,C4.5+m,C4.5+m+cf,22,0.327826
5,C4.5+cf,C4.5+m+cf,10,0.0229091


In [13]:
from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

In [14]:
reject, p_corrected, a1, a2 = multipletests(w_stat['p-value'], alpha = 0.05, method = 'holm')

In [15]:
w_stat['p_corrected'] = p_corrected
w_stat['reject'] = reject

In [16]:
w_stat

,Model 1,Model 2,Wilcoxon stat,p-value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.0107571,0.0645428,False
1,C4.5,C4.5+cf,43,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11,0.0159064,0.0795322,False
3,C4.5+m,C4.5+cf,17,0.0463327,0.138998,False
4,C4.5+m,C4.5+m+cf,22,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10,0.0229091,0.0916364,False


In [18]:
reject, p_corrected, a1, a2 = multipletests(w_stat['p-value'], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [19]:
w_stat['p_corrected'] = p_corrected
w_stat['reject'] = reject

In [20]:
w_stat

,Model 1,Model 2,Wilcoxon stat,p-value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.0107571,0.0458182,True
1,C4.5,C4.5+cf,43,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11,0.0159064,0.0458182,True
3,C4.5+m,C4.5+cf,17,0.0463327,0.0694991,False
4,C4.5+m,C4.5+m+cf,22,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10,0.0229091,0.0458182,True
